In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "at000-hello-acp"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

Task(id='74a31c55-5e45-4f9c-8f47-bf7f38d32c82', created_at=datetime.datetime(2025, 7, 28, 4, 1, 52, 55193, tzinfo=TzInfo(UTC)), name='56a28d29-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 7, 28, 4, 1, 52, 55193, tzinfo=TzInfo(UTC)))


In [4]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Hello what can you do?"},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='8819190c-045a-4baf-b576-3238d815e037', agent_id='1f6cd429-4e4a-4884-b449-72a1f2740393', sequence_id=251, task_id='74a31c55-5e45-4f9c-8f47-bf7f38d32c82', content=TextContent(author='user', content='Hello what can you do?', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 28, 4, 1, 52, 532616, tzinfo=TzInfo(UTC)))


In [5]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=5,
)

╭───────────────────────── USER [07/28/2025 04:01:52] ─────────────────────────╮
│ Hello what can you do?                                                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/28/2025 04:01:52] ─────────────────────────╮
│ Hello! I've received your message. I can't respond right now, but in future  │
│ tutorials we'll see how you can get me to intelligently respond to your      │
│ message.                                                                     │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 5 seconds - returning collected messages
